# word2vec

- 통계 기반 기법의 문제점
  - SVD를 사용하여 차원축소를 했었는데, nxn 행렬에 SVD를 적용하는 비용으ㄴ $O(n^3)$으로 너무 큼.
- 추론 기반 기법
  - neural network가 미니배치로 학습 데이터를 소량씩 나눠서 학습하듯이 추론 기반 기법도 학습 데이터 일부를 순차적으로 학습
  <br/> <img src='../figs/fig%203-1.png'>


### 추론 기반 기법 개요
- 추론 기반 기법은 주변 단어(맥락)이 주어졌을때 무슨 단어가 들어갈지 추측하는 작업.
<br/> <img src='../figs/fig%203-2.png'>

### 신경망에서의 단어 처리
- 신경망을 이용하기 위해 '단어'를 '고정 길이의 벡터'로 변환해야 함.
- One-hot 벡터로 변환
 

In [1]:
import numpy as np
# ID가 0인 벡터를 one-hot 벡터로 표현한 후 fully-conn'd layer를 통과시켜 변환하는 모습
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
h = np.matmul(c, W)
print(h)        

[[1.22704086 1.19957799 0.07702428]]


- 위 식에서 c와 W의 곱은 가중치 W의 row 벡터 하나를 뽑아낸 것과 같음
- 벡터와 행렬을 곱하지 않아도 row 벡터 추출 가능. -> 계산량 감소

In [4]:
# 우리가 구현한 MatMul을 통해 표현하는 방법
import sys
sys.path.append('..')
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[ 1.17488791  0.42578862 -1.25736446]]


## CBOW(Continuous Bag-Of-Words)
- CBOW 모델은 맥락으로부터 target을 추측하는 용도의 신경망.
- input : 맥락, 맥락을 one-hot encoding
- CBOW 구조
<br/> <img src='../figs/fig%203-9.png' width ='300px'>
- 입력시킬 단어의 개수만큼 입력층이 존재.
- hidden layer에서는 input layer들의 평균값 계산
- 출력 시 softmax를 통해 확률 얻음

In [8]:
# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# layer 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# forward propagation
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[ 0.48953878  0.49577206 -0.24207979 -0.58505024  0.2347532  -0.67976034
   0.09603103]]


### 학습
- CBOW 모델은 학습시 사용한 말뭉치(corpus)에 따라 얻게 되는 단어의 분산 표현이 다름.
<br/> <img src='../figs/fig%203-13.png'>
- Softmax로 해당 단어가 해당 자리에 나타날 확률을 구한 후 Cross-entropy Error를 통해 Loss 계산, 학습

### 가중치
- CBOW 모델에서 사용되는 신경망에는 가중치가 2개 존재 : W_in, W_out
- W_in의 각 행이 각 단어의 분산표현임.
- W_out에도 단어의 가중치가 저장됨
<br/> <img src='../figs/fig%203-15.png'>
- word2vec에서는 보통 W_in만 사용함(특히 skip-gram)
- GloVE에서는 두 가중치를 더해서 사용했을때 좋은 결과를 얻었음

In [9]:
from util.corpus import preprocess
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [10]:
print(corpus)

[0 1 2 3 4 1 5 6]


In [12]:
# 맥락(context)과 타깃(target)을 구하는 함수 구현
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    contexts = []
    
    for i in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
        cs.append(corpus[i + t])
        contexts.append(cs)
    
    return np.array(contexts), np.array(target)

In [13]:
contexts, target = create_contexts_target(corpus, window_size=1)

In [14]:
target

array([1, 2, 3, 4, 1, 5])

In [15]:
def convert_one_hot(corpus, vocab_size):
    '''원핫 표현으로 변환

    :param corpus: 단어 ID 목록(1차원 또는 2차원 넘파이 배열)
    :param vocab_size: 어휘 수
    :return: 원핫 표현(2차원 또는 3차원 넘파이 배열)
    '''
    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot

In [16]:
vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [19]:
class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size

        # initialize weight
        W_in = 0.01 * np.random.randn(V, H).astype('f') 
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        # layer 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()
        
        #list에 모으기
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # 단어의 분산표현 저장
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:,0])
        h1 = self.in_layer1.forward(contexts[:,1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss
    
    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None